## Conventions
```
    Python PEP8 Convention + Google Docstring + 개인적인 Convention  
    주석의 첫 단어 사이에는 항상 한칸의 빈칸이 위치해야한다.  
    in-line 주석은 두 칸을 띄어야한다.  
    꼭 필요한 주석인지에 대해 의구심을 품어야한다.  
    Class는 CamelCase
    function은 snake_case
    들여쓰기는 스페이스 네 칸
    [Docstring]
        @ content
        & values description
        $ comment
        ~ example
        <T> type
        - 안쪽에서 사용시 들여쓰기
    
```

## EEG Research Code 

```python
    '''
    info: 
        @ Name: MinGeon Park (Bachelor's 3rd year)
        @ Position: undergraduate researcher
        @ Affiliation: Dept. of Software, Dongseo University ICaN Lab
        @ Email: devmin7985@gmail.com
        @ Github: https://github.com/DevMinGeonPark?tab=repositories
    '''
```

### Setup

In [13]:
'''
    @default-path setup
'''
import os
default_path = '/home/devmin/Desktop/source/jupyter/research-EEG'
print(default_path)
print(os.chdir(default_path))
del default_path

'''
    @imoprt segments
'''
from scipy import io
import numpy as np

# 필요시 주석 해제
#import matplotlib as mpl
#import matplotlib.pylab as plt



/home/devmin/Desktop/source/jupyter/research-EEG
None


### Data Reconstruction

In [14]:
class EEGData:
    '''
        @ Name : EEGData Object
        @ Description : 
            $ EEG Data를 저장하는 객체, Reconstrucion 처리도 진행.
    '''
    def __init__(self, label: str, mat_num: int):
        '''
            @ Name : __init__
            & label :
                <T> str
                ~ sb
                ~ sg
                $ sg, sb data를 구별하여 사용하기 위한 변수
            & mat_num :
                <T> int
                ~ range(1,12)
                $ 1에서부터 11까지 데이터를 받아오기 위한 구별 매개변수
        '''
        self.label = label
        self.mat_num = mat_num
        
        # data_reconstruction is executed only through init
        self.__data_reconstruction()

    
    def __data_reconstruction(self):
        '''
            @ Name : __data_reconstruction
            @ Description :
                -@ Purpose : 
                    $ create 3-dimensional ndarray = [trials x channels x time-series]
                    ~ [30 x 64 x 30720]
                -@ loadmat path :
                    ~ ./data/EEG/{label}/{label}{mat_num}_eeg
                    $ label 변수가 str에서 벗어나지 않게 형식 지정
                -@ Values : 
                    & data[0][0][1] 
                        <T> ndarray (2-dimension/float32)
                        $ It same eeg_data
                        $ 68 x unknown
                    & data[0][0][0][0]
                        <T> ndarray (1-dimension/int)
                        $ It same eeg_event
                        $ 1 is start event
                    & EEG
                        <T> ndarray (2-dimension/float32)
                        $ eeg_data shape is [68 x unknown] like [channel * measured-data]
                        --@ Channels
                            ~ EEG is 1 ~ 64
                            ~ EOG is 65 ~ 66
                            ~ ECG is 67 ~ 68
                        $ np.where(data[0][0][0][0] == 1)[0][0] is start event time
                    & SR
                        <T> int
                        $ srate (Called S-rate)
                        ~ 512
                    & MeasurementTime
                        <T> int
                        $ Measurement Time
                        $ length of EEG[0] normally 921600
                        $ normally 30 between 40
                        --@ formula
                            (len(EEG)/SR)/60
                    & ProcessedData
                        <T> ndarray (3-dimension/flaot32)
                        $ The purpose of the class
                        ~ [30 * 64 * 921600]
                    
                    & tp_startPt
                        <T> int
                        $ start of index about reconstruction
                        --@ formula
                            n x (sr x 60)
                        ~ 15360
                    & tp_endPt
                        <T> int
                        $ end of index about reconstruction
                        --@ formula
                            (SR x 60) + n x (SR x 60)
                        ~ 15360
                        
        '''
        # Meaningful Values
        data = io.loadmat(f'./data/EEG/{str(self.label)}/{str(self.label)}{str(self.mat_num)}_eeg')['eeg']
        EEG = data[0][0][1][:64, np.where(data[0][0][0][0] == 1)[0][0]:]
        SR = data[0][0][-1][0][0]
        MeasurementTime = int((len(EEG[0])/SR)/60)
        # Purpose values
        ProcessedData = np.zeros([MeasurementTime,64,30720])
        
        for temp_trialNum in range(0,MeasurementTime):
            tp_startPt = temp_trialNum*(SR*60)
            tp_endPt = (SR*60) + temp_trialNum*(SR*60)
            # reconstrction
            ProcessedData[temp_trialNum, :, :] = EEG[:, tp_startPt:tp_endPt]
        
        # self Values
        self.SR = SR
        self.ProcessedData = ProcessedData
        
    # get methods
    def get_title(self):
        print(f"현재 데이터 : {self.label}_{self.mat_num}")
    
    def get_SR(self):
        return self.SR

In [16]:

'''
    $ Data Reconstruction from global scope
'''
person_dict = {}
for mat_num in range(1,12):
    sb_data = EEGData('sb',mat_num)
    sg_data = EEGData('sg',mat_num)

    if len(sb_data.ProcessedData) < len(sg_data.ProcessedData): #  like 32
        trials = len(sb_data.ProcessedData)
    else:
        trials = len(sg_data.ProcessedData)
    
    restructured_data = np.zeros([trials*2,64,30720])

    restructured_data[:trials,:,:] = sb_data.ProcessedData[:trials,:,:]
    restructured_data[trials:,:,:] = sg_data.ProcessedData[:trials,:,:]

    person_dict[f"p{mat_num}"] = restructured_data
    del sb_data, sg_data, trials, restructured_data

print(person_dict.keys())

dict_keys(['p1', 'p2', 'p3', 'p4', 'p5', 'p6', 'p7', 'p8', 'p9', 'p10', 'p11'])
